In [1]:
import cv2
import os
import numpy as np
from PIL import Image

In [3]:
def create_user(f_id,name):
    cam = cv2.VideoCapture(0)
    cam.set(3, 640)  # Set video width
    cam.set(4, 480)  # Set video height
    faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    user_dir = f"dataset/user"
    if not os.path.exists(user_dir):
        os.makedirs(user_dir)
    sample_count = 0    
    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to capture image")
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces_detected = faces.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
        for (x, y, w, h) in faces_detected:
            sample_count += 1
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.imwrite(f"{user_dir}/{name}.{f_id}.{sample_count}.jpg", gray[y:y + h, x:x + w])
            cv2.imshow('Face Capture', frame)



        if cv2.waitKey(100) & 0xFF == ord('q') or sample_count >= 80:
            break
    cam.release()
    cv2.destroyAllWindows()


In [17]:
create_user(1, "Robin")

KeyboardInterrupt: 

In [23]:
create_user(3, "arif")

Trainning Face Recognizer

In [24]:
def train_face_recognizer():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    faces = []
    labels = []
    user_dir = "dataset/user"

    for img_name in os.listdir(user_dir):
        img_path = os.path.join(user_dir, img_name)

      
        if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        # Load and convert image to grayscale
        img = Image.open(img_path).convert('L')
        img_np = np.array(img, 'uint8')

        try:
            label = int(img_name.split('.')[1])
        except (IndexError, ValueError):
            print(f"Skipping invalid file name: {img_name}")
            continue

        faces.append(img_np)
        labels.append(label)

    recognizer.train(faces, np.array(labels))
    recognizer.save('face_recognizer.yml')
    print("Training complete and model saved as 'face_recognizer.yml'")
    print(f"Total faces trained: {len(np.unique(labels))}")

In [25]:
train_face_recognizer()

Training complete and model saved as 'face_recognizer.yml'
Total faces trained: 1


In [21]:
def recognize_faces():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('face_recognizer.yml')
    cam = cv2.VideoCapture(0)
    faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to capture image")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces_detected = faces.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces_detected:
            face_roi = gray[y:y + h, x:x + w]
            label, confidence = recognizer.predict(face_roi)
            if confidence < 100:
                if label == 1:
                    name = "Robin"
                elif label == 2:
                    name = "Sifat"
                else:
                    name = f"User {label}"
            else:
                name = "Unknown"
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(frame, name, (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        cv2.imshow('Face Recognition', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cam.release()
    cv2.destroyAllWindows()

In [26]:
recognize_faces()